In [3]:
# Initialize Otter
import otter
grader = otter.Notebook("cs1090a_hw1.ipynb")

# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS1090A Introduction to Data Science
## Homework 1: Webscraping, Pandas, and EDA

**Harvard University**<br/>
**Fall 2025**<br/>
**Instructors**: Pavlos Protopapas, Kevin Rader, and Chris Gumb

<hr style="height:2.4pt">

In [4]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import pyarrow as pa # optional
import seaborn as sns
import matplotlib.pyplot as plt
import re
import io
import os.path
import json
import time
import glob
import asyncio # optional
import aiohttp # optional
from IPython.core.display import HTML
from IPython import display

import wikipedia
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import asyncio
from collections import defaultdict

In [5]:
# For calculating total notebook runtime
notebook_start = time.time()

In [6]:
style = '''<style>
h3, h4 {
    background-color: #7efcf5;
    border-left: 5px solid #7ec4fc;
    border-right: 5px solid #7ec4fc;
    padding: 0em;
}
h3 {
    background-color: #7efcf5;
    border-top: 5px solid #7ec4fc;
    border-left: 5px solid #7ec4fc;
    border-right: 5px solid #7ec4fc;
    padding: 0.5em;
}
p {
    padding: 0.5em;
    max-width: 34em;
    font-weight:400;
}
.md {
    max-width: 80ch;

}
.prompt {    
    background-color: lightgreen;
    border-color: #dFb5b4;
    border-left: 5px solid #f57efc;
    padding: 0.5em;
    font-weight:500;
    }
 </style>'''
HTML(style)

## Assignment Overview

You'll be working with data from screenboston.com, a website that aggregates movie screenings from various theaters in the Boston area. Your task is to collect this data, enrich it with information from Wikipedia, and then analyze and visualize the results.

## Assignment Structure

The assignment is divided into 8 main questions, each building upon the previous ones, followed by a wrap-up:

1. Fetching HTML from screenboston.com (10 pts)
2. Parsing HTML into structured data (15 pts)
3. Augmenting data with historical snapshots (15 pts)
4. Creating and manipulating a Pandas DataFrame (15 pts)
5. Initial exploratory data analysis and visualization (15 pts)
6. Finding Wikipedia pages for each screened film (7 pts)
7. Storing Wikipedia HTML in the DataFrame (3 pts)
8. Extracting and analyzing data from Wikipedia pages (15 pts)
9. Wrap-up (5 pts)

As you progress through these steps, you'll be transforming raw web data into a rich dataset ready for analysis. By the end, you'll have created a comprehensive overview of Boston's movie screening landscape, potentially uncovering interesting trends and patterns in the process. 

<div style = "background: lightgreen; border: thin solid black; border-radius: 2px; padding: 5px">

### Instructions
- To submit your notebook, follow the instructions given in on the Canvas assignment page.
- Plots should be legible and interpretable *without having to refer to the code that generated them*. They should include labels for the $x$- and $y$-axes as well as a descriptive title and/or legend when appropriate.
- When asked to interpret a visualization, do not simply describe it (e.g., "the curve has a steep slope up"), but instead explain what you believe the plot *means*.
- Autograding tests are mostly to help you debug. The tests are not exhaustive so simply passing all tests may not be sufficient for full credit.
- The use of *extremely* inefficient or error-prone code (e.g., copy-pasting nearly identical commands rather than looping) may result in only partial credit.
- We have tried to include all the libraries you may need to do the assignment in the imports cell provided below. Please get course staff approval before importing any additional 3rd party libraries.
- Enable scrolling output on cells with very long output. Or better yet, avoid overly long output altogether.
- Feel free to add additional code or markdown cells as needed.
- Don't forget to include a written response when one is requested by a question prompt.
- Ensure your code runs top to bottom without error and passes all tests by restarting the kernel and running all cells (note that this can take a few minutes). 
- **You should do a "Restart Kernel and Run All Cells" before submitting to ensure (1) your notebook actually runs and (2) all output is visible**
</div>


<div class=md>
<h3>1. Getting HTML from screenboston.com</h3>

<p>Our first step is to get the HTML content from the page at screenboston.com for analysis. To avoid having to make unnecessary future requests, you should write the content to disk.</p>



<div class=prompt>
    
1. Use the `requests` library to get the html located at screenboston.com.
1. Save it in the file `data/html/screenboston.html`.

**Your code should only make an HTTP request if the file does not already exist.** 

**Hint:** the `os.path.isfile()` function can be used check if a file exists.
</div>
</div>


In [17]:
SCREEN_BOSTON_URL = "https://screenboston.com"

In [7]:
# your code here

import requests
path = "data/html/screenboston.html"
os.makedirs(os.path.dirname(path), exist_ok=True)

if not os.path.isfile(path):
    with open(path, "w", encoding="latin1") as f:
        #save text attribute from request response to get the html content explicitly
        f.write(requests.get(SCREEN_BOSTON_URL).text)
with open(path, "r") as f:
    content = f.read()

print(content)

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, maximum-scale=1" />
    <link href="./_app/immutable/assets/0.d965597f.css" rel="stylesheet">
    <title>Screen Boston</title><!-- HEAD_svelte-i2fggh_START -->
    <meta name="title" content="Screen Boston">
    <meta property="og:title" content="Screen Boston">
    <meta name="twitter:title" content="Screen Boston">
    <meta name="description" content="Explore repertory and independent film screenings in the Boston area. Find movies playing in arthouse theaters near you: film festivals, special events, &amp; more.">
    <meta property="og:description" content="Explore repertory and independent film screenings in the Boston area. Find movies playing in arth
    ouse theaters near you: film festivals, special events, &amp; more.">
    <meta name="twitter:description" content="Explore repertory and independent film screenings in the Boston area. Find movies playin

In [8]:
content

'<!DOCTYPE html>\n<html lang="en">\n\n<head>\n    <meta charset="utf-8" />\n    <meta name="viewport" content="width=device-width, maximum-scale=1" />\n    <link href="./_app/immutable/assets/0.d965597f.css" rel="stylesheet">\n    <title>Screen Boston</title><!-- HEAD_svelte-i2fggh_START -->\n    <meta name="title" content="Screen Boston">\n    <meta property="og:title" content="Screen Boston">\n    <meta name="twitter:title" content="Screen Boston">\n    <meta name="description" content="Explore repertory and independent film screenings in the Boston area. Find movies playing in arthouse theaters near you: film festivals, special events, &amp; more.">\n    <meta property="og:description" content="Explore repertory and independent film screenings in the Boston area. Find movies playing in arth\n    ouse theaters near you: film festivals, special events, &amp; more.">\n    <meta name="twitter:description" content="Explore repertory and independent film screenings in the Boston area. Fin

In [9]:
grader.check("q1")

q1 results: All test cases passed!

<div class=md>
<h3>2. Parsing HTML into an Efficient Data Structure</h3>

<p>Were you to open this HTML file in an editor, you'd find it wouldn't make for light reading.
There's a lot of text in there, but we're currently only interested in a small portion of it. Thankfully, this small portion's constituant occurances appear in a mostly predictable way.
</p>

<div class='prompt'>
1. Parse the HTML page to produce an iterable of dictionaries called <code>movies</code>. Here is one possible example:
<div style="max-width:30em;padding:1em">
    
```python
{'title': 'Monty Python and the Holy Grail',
 'directors': 'Terry Jones, Terry Gilliam',
 'year': 1975,
 'genre': 'Adventure',
 'runtime': '1h 31m',
 'theater': 'Coolidge Corner Theater',
 'screen_date': '2024-08-17'
 'screen_times': '11:59 PM'}
```
</div>

2. Report the number of dictionaries in your iterable.
3. Display an example dictionary.
</div>

<p>
<b>One possible approach:</b> leveraging the structure of the page, use BeautifulSoup to divide it into disjoint peices of the HTML tree, each containing the set of information outlined above for each title's screening date at a given theater. Each peice is processed by a helper function(s) which extracts the desired fields and assemples them into a dictionary. These dictionaries are then accumulated in some iterable such as a list.
</p>

**Hints:**
- Your browser's 'inspect' tool is very useful for exploring a page's structure.
- To maintain a uniform set of keys across all movie dictionaries, use the plural 'directors' even for movies which attribute only a single director. 
</div>
</div>

In [10]:
from bs4 import BeautifulSoup
import requests

In [11]:
soup = BeautifulSoup(content, "html.parser")

In [12]:
for div in soup.find_all("p",class_="big"):
    print(div.parent.parent.parent.parent.parent.prettify())

<div class="max-w-screen grid lg:grid-cols-2 gap-0.5 p-0.5 px-1 pb-2 z-0 md:col-span-4 lg:col-span-2">
 <button class="w-full h-auto max-w-full text-left">
  <div class="grid h-full md:grid-cols-5 gap-1 bg-white hover:bg-white/60 transition-colors p-1">
   <div class="md:col-span-2">
    <div class="aspect-w-16 aspect-h-9">
     <div class="bg-grey w-full h-full">
      <img alt="Clueless" class="w-full h-full object-cover opacity-0 duration-600 transition-opacity ease-in" src="https://img.screenboston.com/eyJidWNrZXQiOiJzY3JlZW5ib3N0b25waG90b3MiLCJrZXkiOiJlY2QwMzAyMi1mYzkyLTRjYTktOGIzYi1lN2UyYzFhYzRlZmFfMTUwMHgxMDAwLTEuanBnIiwiZWRpdHMiOnsicmVzaXplIjp7IndpZHRoIjoxMDYwLCJmaXQiOiJjb3ZlciJ9fX0=" srcset="https://img.screenboston.com/eyJidWNrZXQiOiJzY3JlZW5ib3N0b25waG90b3MiLCJrZXkiOiJlY2QwMzAyMi1mYzkyLTRjYTktOGIzYi1lN2UyYzFhYzRlZmFfMTUwMHgxMDAwLTEuanBnIiwiZWRpdHMiOnsicmVzaXplIjp7IndpZHRoIjozMDAsImZpdCI6ImNvdmVyIn19fQ== 300w, https://img.screenboston.com/eyJidWNrZXQiOiJzY3JlZW5ib3N0b25waG90b

In [13]:
 for div in soup.find_all("p", class_="big"):
    print("=" * 40)
    print("CURRENT ELEMENT:")
    print(div.prettify())

    # One level up (parent)
    parent = div.parent
    print("\nPARENT:")
    print(parent.prettify())

    # Two levels up (grandparent)
    grandparent = parent.parent
    if grandparent:
        print("\nGRANDPARENT:")
        print(grandparent.prettify())

    # One level down (children)
    print("\nCHILDREN:")
    for child in div.children:
        print(repr(child))  # repr helps show text nodes clearly

    # Two levels down (grandchildren)
    print("\nGRANDCHILDREN:")
    for child in div.children:
        if hasattr(child, "children"):
            for grandchild in child.children:
                print(repr(grandchild))

CURRENT ELEMENT:
<p class="big whitespace-normal w-full min-w-0 overflow-hidden">
 Clueless
</p>


PARENT:
<div class="w-full min-w-0">
 <p class="big whitespace-normal w-full min-w-0 overflow-hidden">
  Clueless
 </p>
 <div class="mt-[2px]">
  <p class="text-[14px] leading-[17px] text-primary">
   House Lights On for 6:00pm Screening
  </p>
 </div>
</div>


GRANDPARENT:
<div class="md:col-span-3 min-w-0 justify-between gap-4 md:gap-1 flex flex-col w-full sm:h-full">
 <div class="w-full min-w-0">
  <p class="big whitespace-normal w-full min-w-0 overflow-hidden">
   Clueless
  </p>
  <div class="mt-[2px]">
   <p class="text-[14px] leading-[17px] text-primary">
    House Lights On for 6:00pm Screening
   </p>
  </div>
 </div>
 <div class="flex child:flex-1 gap-1 -mb-[3px] md:-mb-[.5px]">
  <div>
   <p class="">
    Amy Heckerling
   </p>
   <p class="">
    1995, Comedy, 1h 37m
   </p>
  </div>
  <div>
   <p>
    The Brattle
   </p>
   <div class="flex gap-[6px]">
    <p>
     6:00 PM
  

In [14]:
movies = []

for div in soup.find_all("p",class_="big"):
    movie = {}
    cleaned_title = div.text.strip()
    movie.update({"title": cleaned_title})

    info_blocks = div.find_parent().find_next_sibling("div").find_all("p")

    info_blocks = [p.text.strip() for p in info_blocks]
    movie.update({"directors": info_blocks[0].strip()})
    misc_parts = info_blocks[1].split(",")
    # print(misc_parts)
    if len(misc_parts) == 3:
        movie.update({"year": int(misc_parts[0].strip())})
        movie.update({"genre": misc_parts[1].strip()})
        movie.update({"runtime": misc_parts[2].strip()})
    else:
        # Skipping appending incomplete records for now
        continue
        
    movie.update({"theater": info_blocks[2].strip()})
    movie.update({"screen_times": info_blocks[3].strip()})

    # Date (from nearest <p class="small"> above card)
    card = div.find_parent().find_parent()
    date_tag = card.find_previous("p", class_="small")
    if date_tag:
        movie.update({"screen_date": date_tag.get_text(strip=True)})

    movies.append(movie)

In [16]:
type (date_tag)

bs4.element.Tag

In [102]:
movies

[{'title': 'Clueless',
  'directors': 'Amy Heckerling',
  'year': 1995,
  'genre': 'Comedy',
  'runtime': '1h 37m',
  'theater': 'The Brattle',
  'screen_times': '6:00 PM',
  'screen_date': 'Monday, September 15'},
 {'title': 'Like You Know It All',
  'directors': 'Hong Sang-soo',
  'year': 2009,
  'genre': 'Drama',
  'runtime': '2h 8m',
  'theater': 'Harvard Film Archive',
  'screen_times': '7:00 PM',
  'screen_date': 'Monday, September 15'},
 {'title': 'Postcards from the Edge',
  'directors': 'Mike Nichols',
  'year': 1990,
  'genre': 'Comedy',
  'runtime': '1h 41m',
  'theater': 'Coolidge Corner Theatre',
  'screen_times': '7:00 PM',
  'screen_date': 'Monday, September 15'},
 {'title': 'The Rocketeer',
  'directors': 'Joe Johnston',
  'year': 1991,
  'genre': 'Action',
  'runtime': '1h 48m',
  'theater': 'Somerville Theatre',
  'screen_times': '7:30 PM',
  'screen_date': 'Monday, September 15'},
 {'title': 'Twin Peaks: The Return (Part 13 & 14)',
  'directors': 'David Lynch',
  'ye

In [101]:
grader.check("q2")

q2 results: All test cases passed!

<div class=md>
    <h3>3. Adding Historical Data: Snapshots from the Internet Archive's Wayback Machine</h3>
<p>
The screenboston.com page only shows screenings scheduled from the current day until about 2-4 weeks into the future.
Using the <a href="http://web.archive.org/">Wayback Machine</a>, we can augment our dataset with past snapshots of the page  
</p>

<p>
Rather than clicking around to download each snapshot manually, we should again acquire our pages programmatically using <code>requests</code>. The Internet Archive offers an <a href='https://archive.org/help/wayback_api.php'>API</a> where you can specify a URL and timestamp and receive a JSON object which contains a new URL to a snapshot of the desired page nearest to the provided timestamp. You can then use <code>requests</code> again to acquire the HTML of this snapshot.
</p>
    
<div class=prompt>

1. Acquire enough snapshots to cover all days from Jan through September 2024 for which there exists data. 
For simplicity we'll assume that any well-formed screenboston.com page will cover at least through the end of that month and that a day's schedule won't change once displayed.
2. Write each snapshot's HTML content to disk in the `/data/html` directory using filenames of the form `'snapshot_YYYYMMDD.html'`. Your code should only make requests if no local snapshot files already exist.
3. Using your parsing function from the previous section, add movie dictionaries extracted from all the snapshots to your list, `movies`. Save this list of dictionaries as 'data/movies.json'. You should only perform the parsing if the JSON file does not already exist. Don't worry about any duplicate movie screenings in your list at this point.

</div>

**Hints:**
- When retrieving JSON content with <code>requests</code>, us the `json()` method rather than the `text` attribute to extract the content from the returned request object into a nested dictionary rather than a string.
- APIs often take parameters as key-value pairs occuring after a '?' character in the URL. For example 'http://archive.org/wayback/available?url=example.com&timestamp=20060101' includes a 'url' and a 'timestamp' parameter. When using <code>requests.get()</code> you can exclude these parameters from the URL itself and instead pass them as a dictionary to an argument called 'params'.
- Use <code>json.dump()</code> and <code>json.load()</code> from the imported <code>json</code> module for writing and reading JSON content.

##### Structure:

**Func1** - Call wayback api with date
Get json response and call respn.
	data = resp.json()

Get snapshot URL from data
snapshot_url = data["archived_snapshots"]["closest"]["url"]

**Func2** - Call snapshot URL with requests to get html content

Write it to a file in
data/html/snapshot_YYYYMMDD.html

**Func3** - Recursively read html from this folder and call dict function from question 2, then convert all to .json

In [15]:
base_url = "https://archive.org/wayback/available"

timestamp = 20190505
params = {"url": SCREEN_BOSTON_URL, "timestamp": timestamp}
requests.get("https://archive.org/wayback/available?url=example.com&timestamp=20060101")

In [16]:
# Display info about snapshots and results of parsing
print(f"Snapshots:")
print([f.split('/')[-1] for f in glob.glob('data/html/snapshot_*')])
print(f"Found {len(movies)} movie entries across all HTML files.")

Snapshots:
[]


NameError: name 'movies' is not defined

In [ ]:
grader.check("q3")

<div class=md>
    
<h3>4. DataFrame Creation & Manipulation</h3>

Your variable `movies` is a list of dictionaries which you can use to create a Pandas DataFrame. Have our data in a DataFrame will make it much easier to analyze and manipulate.

<div class=prompt>
    
1. Use `movies` to create a DataFrame and call it `df`
2. Drop any **duplicate rows** and report the number of remaining rows. Here we consider duplicates to be rows in which *all* values match. The same movie screening on different days, even at the same theater, will not be considered a duplicate.
3. Check for **nans**. If you find any nans you should report the number of nans in each column and display the rows that contain at least one nan. If there are no nans then you should display output to confirm this fact.
4. Convert the **'screen_date'** to a Pandas `datetime` dtype and **'runtime'** to a Pandas `timedelta` dtype.
5. Sort your `df` by ascending 'screen_date'
</div>

Your result should look something like:

| title                            | directors       | year | genre  | runtime         | theater                   | screen_date | screen_times         |
|-----------------------------------|-----------------|------|--------|-----------------|---------------------------|-------------|----------------------|
| Ghost Dog: The Way of the Samurai | Jim Jarmusch    | 1999 | Crime  | 0 days 01:56:00 | The Brattle               | 2024-05-24  | 4:00 PM, 9:00 PM     |
| Stray Dog                        | Akira Kurosawa  | 1949 | Crime  | 0 days 02:02:00 | The Brattle               | 2024-05-24  | 6:30 PM              |
| Big Trouble in Little China       | John Carpenter  | 1986 | Comedy | 0 days 01:40:00 | Coolidge Corner Theatre    | 2024-05-24  | 11:59 PM             |
| Young Frankenstein               | Mel Brooks      | 1974 | Comedy | 0 days 01:46:00 | The Brattle               | 2024-05-25  | 12:00 PM             |
| Poetry                           | Lee Chang-dong  | 2010 | Drama  | 0 days 02:19:00 | Museum of Fine Arts        | 2024-05-25  | 2:30 PM              |

**Hints:**
- You can use the `dt.date` attribute on a datetime column to extract just the day (and ignore the time)
- You may need to to do some string parsing on 'runtime' before coverting it to a `timedelta`
- Pandas now supports [PyArrow](https://arrow.apache.org/docs/python/index.html) as a backend alternative to Numpy. This gives access to new datatypes including native strings (as opposed to the catch-all 'object'), ints that support NaN values, and others. Using these datatypes can often speed up computations so you may choose to experiment with converting some or all of your columns to these new datatypes. You can read more [here](https://pandas.pydata.org/docs/user_guide/pyarrow.html).
</div>

In [ ]:
# your code here
...

In [ ]:
...

In [ ]:
grader.check("q4")

<div class=md>
    
<h3>5. Initial Exploratory Data Analysis (EDA)</h3>

Using your DataFrame, perform the following exploratory data analysis tasks. Note that we'll use the term "screening" to refer to rows in the DataFrame example provided above. For each visualization component of this question you should interpret the results and draw conclusions.

<div class=prompt>

1. **Longest Films**:
   Display the rows corresponding to the top 5 longest reported runtimes in descending order.

2. **Screening Counts**:
   Create two plots as side-by-side subplots with a shared y-axis range:\
   a. Number of screenings by theater\
   b. Number of screenings by genre

3. **Movie Year Analysis**:
   Create two separate plots:\
   a. Distribution of 'year' for all movies\
   b. Distribution of 'year' grouped by theater

4. **Custom Analysis**:
   Ask two questions of your choice regarding the screening data and use plots to attempt to find some answers. For each question:\
   a. Clearly state your question\
   b. Explain your approach to answering the question\
   c. Create an appropriate visualization
   
   At least one of these new plots must be a different type from those used in parts 2 and 3.

Example questions you might consider:
- Is there a relationship between a movie's age (current year - release year) and the number of screenings it receives?
- Do certain genres tend to have longer runtimes?
- Are there patterns in screening times for different theaters or genres?
- How does the distribution of genres vary across different theaters?


</div>
</div>

In [ ]:
# 1. Longest films
# your code here
...

In [ ]:
# 2. Screenings by theater & screenings by genre
# your code here
...

In [ ]:
# 3. Year distribution & distribution of year by theater
# your code here
...

In [ ]:
# 4.1 Custom question 1 - Runtime Distributions by Year (Scatter with Trend Line, Outliers Removed but Included in Regression)
# your code here
...


In [ ]:
# 4.2 Custom question 2 - Top 10 Directors by Number of Screenings (Aesthetic Improvements)
# your code here
...


<div class=md>
<h3>6. Get each film’s Wikipedia page identifier</h3>

Use the Wikipedia Python library to find a stable identifier for each film’s Wikipedia page and store it in a new `wiki_id` column. Prefer the numeric page ID (`page.pageid`); a canonical article URL is acceptable. Display the head of your updated DataFrame and briefly describe your approach. Aim for at least 50% coverage.

<strong>Hints:</strong>
- Set up the library: `wikipedia.set_lang("en")` and `wikipedia.set_rate_limiting(True)`.
- Build your search query from known fields (e.g., title + year + "film" [+ director if helpful]).
- Use `wikipedia.search(...)`, then fetch a page with `wikipedia.page(..., auto_suggest=True)`.
- Extract a stable ID: `page.pageid` (preferred) or `page.url`.
- Handle `DisambiguationError` and `PageError`; return `None` when you’re not confident.
- Cache results (e.g., write `wiki_id` to a CSV and only query rows where it’s missing).
- Optional: use a small thread pool for speed, but keep concurrency low to be polite to Wikipedia.


<strong>Optional performance hints (polite concurrency):</strong>
- The <code>wikipedia</code> calls are blocking I/O. If you want to speed things up, use a small <code>ThreadPoolExecutor</code> (threads are appropriate for I/O). Keep <code>max_workers</code> low (2–4; at most ~8) and enable <code>wikipedia.set_rate_limiting(True)</code>.
- Simple threaded map (no asyncio):
  
  ```python
  from concurrent.futures import ThreadPoolExecutor

  wikipedia.set_rate_limiting(True)

  def get_wiki_id_for_movie(row):
      # your blocking lookup; return pageid or None
      ...

  rows = df.to_dict("records")
  with ThreadPoolExecutor(max_workers=3) as ex:
      wiki_ids = list(ex.map(get_wiki_id_for_movie, rows))
  df["wiki_id"] = wiki_ids
  ```
- If you prefer <code>asyncio</code>, you can run the blocking function in a thread pool and gather results:
  
  ```python
  import asyncio
  from concurrent.futures import ThreadPoolExecutor

  async def get_wiki_ids_async(rows, max_workers=3):
      loop = asyncio.get_running_loop()
      with ThreadPoolExecutor(max_workers=max_workers) as ex:
          tasks = [loop.run_in_executor(ex, get_wiki_id_for_movie, r) for r in rows]
          return await asyncio.gather(*tasks)

  wiki_ids = await get_wiki_ids_async(df.to_dict("records"), max_workers=3)
  df["wiki_id"] = wiki_ids
  ```
- Cache results to disk so you don’t repeat requests. Handle <code>DisambiguationError</code>/<code>PageError</code> and return <code>None</code> gracefully.


<strong>Ensure that :</strong>
- `df` has a `wiki_id` column.
- ≥50% of `wiki_id` are non-null.
- You show the head of the updated DataFrame and briefly describe your approach.

</div>

In [ ]:
# Custom question 2 - Retrieving Wikipedia page IDs for movies
...

In [ ]:
# Display results of updated df
missing_rows = df[df.wiki_id.isna()]
num_wiki_id_na = missing_rows.shape[0]
print("Number of screenings missing a wiki ID:", num_wiki_id_na)
if num_wiki_id_na > 0:
    print("Example rows with missing wiki ID")
    display.display(missing_rows)

*your answer here*

With these missed pages the above approach is still suboptimal.
The code makes aysnyc wikipedia searches of the form '\<title> \<director> film'. It then goes through the search result pages until it finds one in which half of its 'categories' (listed at the bottom of a Wikipedia page) are contain 'film'. It probably would have been better to check for certain fields in the infobox (the section on the top right of most wikipedia pages) rather than the looking at the categories.

In [ ]:
grader.check("q6")

<div class='md'>
    
<h3>7. Acquire HTML from wiki page and store in DataFrame</h3>

<div class=prompt>
    
Use `'wiki_id'` to acquire the HTML content of each film's Wikipedia page and store it in a new column called `'wiki_html'`.
</div>
</div>

In [ ]:
...

In [ ]:
# Display results of updated df
missing_rows = df[df.wiki_html.isna()]
num_wiki_html_na = missing_rows.shape[0]
print("Number of screenings missing wiki HTML:", num_wiki_html_na)
if num_wiki_html_na > 0:
    print("Example rows with missing wiki HTML")
    display.display(missing_rows)

In [ ]:
grader.check("q7")

<div class=md>

<h3>8. Adding Data from Wikipedia and Asking New Questions</h3>

Now that we have Wikipedia pages for most or all of our movies, we can add supplemental data from those pages to our DataFrame, ask new questions, and use Pandas and visualizations to try and get some answers. This final section is very open-ended, and you are encouraged to experiment. We want to see what you can do!

<div class='prompt'>

1. **Data Extraction and Integration**
   - Extract at least one new piece of data from each movie page you found.
   - Add this data as a new column(s) to your DataFrame.
   - Display the head of your updated DataFrame.
   - Discuss your strategy for extracting the new data, including any challenges you faced and how you overcame them.

2. **Data Analysis and Visualization**
    - Formulate at least two new questions that involve the Wikipedia data you've added.
    - For each question:
      - Clearly state your question and why it's interesting or relevant.
      - Describe your approach to answering the question.
      - Use Pandas to manipulate and analyze the data as needed.
      - Create at least one meaningful visualization that helps answer your question.
      - Ensure your plots are well-labeled and easy to interpret.
      - Interpret the results of your analysis, discussing any insights, patterns, or surprises you discovered.

</div>

**Hints:**
- The 'info box' on the right side of most Wikipedia pages is fairly standardized across film pages. This might be the easiest place to start.
- Some potential pieces of data to extract include the language of the film, its box office revenue or budget, the starring cast, etc.
- Feel free to extract more than just a single new piece of data to use in your analysis. If you're feeling particularly adventurous, you can even jump from the film page to other linked pages to acquire more data!
- Consider how the new data you've extracted might relate to the screening information you already have. Are there interesting connections or patterns to explore?

Remember, the goal is to demonstrate your ability to ask insightful questions, use appropriate data science techniques to answer them, and effectively communicate your findings.
</div>

In [ ]:
# 1. Data Extraction and Integration
# your code here
...

In [ ]:
# 2. Data Analysis and Visualization
# your code here
...

In [ ]:
# your code here
...

In [ ]:
# your code here
...

In [ ]:
# your code here
...

<div class="alert alert-info" style="color: #4a4a4a; background-color: #fbe8ff; border-color: #eed4db; border-width: 1px; border-radius: 3px; padding: 10px;">

**Wrap-up**

* Please describe the aspect(s) of the assignment you found most challenging. This could be conceptual and/or related to coding and implementation. This is also a good place to ask any remaining questions you may have.

* How many hours did you spend working on this assignment? Store this as an int or float in `hours_spent_on_hw`. If you worked on the project in a group, report the *average* time spent per person.

</div>

In [ ]:
hours_spent_on_hw = ...

In [ ]:
grader.check("wrapup")

In [ ]:
notebook_end = time.time()
print(f"It took {(notebook_end - notebook_start)/60:.2f} minutes for this notebook to run")

🌈 **This concludes HW1. Thank you!**